In [1]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import Annotated
from typing_extensions import TypedDict
from operator import add
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
class SharedState(TypedDict):
    dsa_topics: list[str]
    system_design_topics: list[str]

def parse_model_response(response):
    """ Parse the model response"""
    return [topic.strip() for topic in response.split(",") if topic.strip()]

In [4]:
def find_relevant_dsa_topics(shared_state: SharedState) -> SharedState:
    """ Find relevant DSA topics for year 2025."""
    query = """
    Can you provide top 5 DSA topics to master for Software Engineering interviews in 2025?.
    Please only return the DSA topics comma separated, no other detail is needed. 
    """
    model = ChatOpenAI(model="gpt-4o")
    
    response = model.invoke(query)
    # shared_state["dsa_topics"] = parse_model_response(response.content)

    return {
        "dsa_topics": parse_model_response(response.content),
        "system_design_topics": []
    }

In [12]:
def find_relevant_system_design_topics(shared_state: SharedState) -> SharedState:
    """ Find relevant System Design topics for year 2025."""
    query = """
    Can you provide top 5 System Design topics to master for Software Engineering interviews in 2025?.
    Please only return the System Design topics comma separated, no other detail is needed. 
    """
    model = ChatOpenAI(model="gpt-4o")
    
    response = model.invoke(query)
    shared_state["system_design_topics"] = parse_model_response(response.content)

    return shared_state

In [6]:
def build_graph():
    # Building a Graph
  # State of the Graph that will be shared among nodes.
  workflow = StateGraph(SharedState)

  # Add nodes.
  workflow.add_node("find_relevant_dsa_topics", find_relevant_dsa_topics)
  workflow.add_node("find_relevant_system_design_topics", find_relevant_system_design_topics)

  # Define the edges of the graph.
  workflow.add_edge(START, "find_relevant_dsa_topics")
  workflow.add_edge("find_relevant_dsa_topics", "find_relevant_system_design_topics")
  workflow.add_edge("find_relevant_system_design_topics", END)

  checkpointer = InMemorySaver()
  graph = workflow.compile(checkpointer=checkpointer)

  config = {"configurable": {"thread_id": "1"}}
  response = graph.invoke({}, config)

  # print(graph.get_graph().draw_mermaid())

  return response, graph, config

In [13]:
response, graph, config = build_graph()
print(response)

{'dsa_topics': ['Arrays', 'Strings', 'Trees', 'Graphs', 'Dynamic Programming'], 'system_design_topics': ['Scalability and Load Balancing', 'Data Storage and Management', 'Microservices Architecture', 'High Availability and Fault Tolerance', 'Real-Time Data Processing.']}


In [14]:
snapshots = list(graph.get_state_history(config))

In [15]:
print(snapshots[1])

StateSnapshot(values={'dsa_topics': ['Arrays', 'Strings', 'Trees', 'Graphs', 'Dynamic Programming'], 'system_design_topics': []}, next=('find_relevant_system_design_topics',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0621d7-4614-6ed8-8001-15c409bf22f5'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}, 'thread_id': '1'}, created_at='2025-07-16T08:18:29.459290+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0621d7-3c11-6b2a-8000-dc2ec3bde44b'}}, tasks=(PregelTask(id='200ba79c-7371-04c9-3118-9437577f3cce', name='find_relevant_system_design_topics', path=('__pregel_pull', 'find_relevant_system_design_topics'), error=None, interrupts=(), state=None, result={'dsa_topics': ['Arrays', 'Strings', 'Trees', 'Graphs', 'Dynamic Programming'], 'system_design_topics': ['Scalability and Load Balancing', 'Data Storage and Management', 'Microservices Architecture', 'High Availability and Fault Tolerance'

In [16]:
list.reverse(snapshots)

for snapshot in snapshots:
    print("================== Snapshot details: ==================")
    print(snapshot.values)
    print(snapshot.next)
    print("\n\n")

================== Snapshot details: ==================
{}
('__start__',)



================== Snapshot details: ==================
{}
('find_relevant_dsa_topics',)



================== Snapshot details: ==================
{'dsa_topics': ['Arrays', 'Strings', 'Trees', 'Graphs', 'Dynamic Programming'], 'system_design_topics': []}
('find_relevant_system_design_topics',)



================== Snapshot details: ==================
{'dsa_topics': ['Arrays', 'Strings', 'Trees', 'Graphs', 'Dynamic Programming'], 'system_design_topics': ['Scalability and Load Balancing', 'Data Storage and Management', 'Microservices Architecture', 'High Availability and Fault Tolerance', 'Real-Time Data Processing.']}
()



